In [1]:
%pip install transformers

In [3]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import AutoConfig, AutoModel, AutoTokenizer, get_linear_schedule_with_warmup
import torch.nn.functional as F

In [4]:
class args:
    model = "bert-base-uncased"
    epochs = 20
    batch_size = 24
    learning_rate = 5e-5
    train_batch_size = 24
    valid_batch_size = 24
    max_len = 128
    accumulation_steps = 1
    checkpoint_path="/content/drive/MyDrive/MiniProject/model/model_recent_checkpoint.pth"

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
class TextModel_pred(nn.Module):
    def __init__(self,num_classes):
        super().__init__()
        hidden_dropout_prob: float = 0.1
        layer_norm_eps: float = 5e-5

        config = AutoConfig.from_pretrained(args.model)

        config.update(
            {
                "output_hidden_states": True,
                # "hidden_dropout_prob": hidden_dropout_prob,
                "layer_norm_eps": layer_norm_eps,
                "add_pooling_layer": False,
                "num_labels": 1,
            }
        )
        self.bert=AutoModel.from_pretrained(args.model, config=config)
        self.bert_drop=nn.Dropout(0.1)
        self.fc=nn.Linear(768,num_classes)

    def forward(self,ids,attn_mask,token_id,target=None):
        x=self.bert(ids,attention_mask=attn_mask,token_type_ids=token_id)
        x=x.pooler_output
        #x=self.bert_drop(x)
        x=self.fc(x)
        x=torch.sigmoid(x)
        # x=F.softmax(x, dim=1)
        return x

In [21]:
model_pred = TextModel_pred(
    # model_name=args.model,
    num_classes=1,
)
model_pred.load_state_dict(torch.load(args.checkpoint_path))
model_pred.to("cuda")

TextModel_pred(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=5e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=5e-05, elementwise_af

In [22]:
#  text=input("Enter the text")
def pred(text):
  def tokenn(text,max_len=256):
          tokenizer=AutoTokenizer.from_pretrained(
              "bert-base-uncased",
              do_lower_case=False
          )
          tokens=tokenizer(
              text,
              None,
              add_special_tokens=True,
              max_length=max_len,
              padding='max_length',
              truncation=True
          )

          return {
              "ids":torch.tensor(tokens["input_ids"],dtype=torch.long),
              "attn_mask":torch.tensor(tokens["attention_mask"],dtype=torch.long),
              "token_id":torch.tensor(tokens["token_type_ids"],dtype=torch.long),
          }

  params=tokenn(text)
  ids=params['ids'].view(1,256)
  attn_mask=params['attn_mask'].view(1,256)
  token_id=params['token_id'].view(1,256)
  ids,attn_mask,token_id=ids.to("cuda"),attn_mask.to("cuda"),token_id.to("cuda")
  output=model_pred.forward(ids,attn_mask,token_id)
  if output<0.5:
    return "Non toxic"
  else:
    return "Toxic"

In [23]:
pred("hello friengs")

'Non toxic'